In [2]:
import re
import pandas as pd
import os
import subprocess
import tempfile
from io import StringIO

In [10]:
data_location = "/home/t/ticeraskin/Desktop/project/Remuszka_Shared/GovRnD/data/"

In [4]:
def extract_with_system_tools(zip_path, outdir):
    from shutil import which

    # Try 7zip first (handles all ZIP compression methods)
    for exe in ["7z", "7za", "7zr"]:
        seven = which(exe)
        if seven:
            cmd = [seven, "x", zip_path, f"-o{outdir}", "-y"]
            try:
                subprocess.run(cmd, check=True)
                print(f"Extracted with {exe}.")
                return True
            except subprocess.CalledProcessError:
                pass

    # Try unzip (may or may not support your method)
    unzip = which("unzip")
    if unzip:
        try:
            subprocess.run([unzip, "-o", zip_path, "-d", outdir], check=True)
            print("Extracted with unzip.")
            return True
        except subprocess.CalledProcessError:
            pass

    raise RuntimeError("No extractor available (7z/7za/7zr/unzip not found).")


def load_zip_as_dataframes(zip_path):
    # Temporary directory for extraction
    with tempfile.TemporaryDirectory() as temp_dir:

        # Extract using 7-Zip or unzip
        extract_with_system_tools(zip_path, temp_dir)

        dfs = {}

        # Walk extracted files
        for root, dirs, files in os.walk(temp_dir):
            for filename in files:
                path = os.path.join(root, filename)

                # Load file contents
                try:
                    with open(path, "r", encoding="utf-8", errors="ignore") as f:
                        text = f.read()
                except Exception:
                    with open(path, "rb") as f:
                        # binary fallback
                        dfs[filename] = pd.DataFrame({"bytes": [f.read()]})
                        continue

                # Try CSV parsing
                buffer = StringIO(text)
                try:
                    df = pd.read_csv(buffer)
                except Exception:
                    # fallback: line-per-row
                    df = pd.DataFrame({"text": text.splitlines()})

                dfs[filename] = df

        return dfs

def merge_yearly_files(dfs, base_name="base_file"):
    """
    dfs: dict {filename: DataFrame}
    base_name: the prefix before _YYYY.csv
    Returns a concatenated DataFrame with an added column 'year'.
    """
    pattern = re.compile(rf"^{base_name}_(\d{{4}})\.csv$")

    frames = []

    for filename, df in dfs.items():
        match = pattern.match(filename)
        if match:
            year = int(match.group(1))
            df = df.copy()
            df["year"] = year
            frames.append(df)

    if not frames:
        raise ValueError(f"No files found matching pattern {base_name}_YYYY.csv")

    return pd.concat(frames, ignore_index=True)


In [5]:
# Bringing in all the contracts
## Reading zip files
dfs = load_zip_as_dataframes(data_location+"G Contracts/base_files.zip")
dfs.keys()

## Putting all contracts into one file
contracts = merge_yearly_files(dfs)

## Removing zeros for size concerns
contracts_nonzero = contracts[contracts['federal_action_obligation']!=0]


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,96 CPUs Intel(R) Xeon(R) Gold 6248R CPU @ 3.00GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
1 file, 3521570299 bytes (3359 MiB)

Extracting archive: /home/t/ticeraskin/Desktop/project/Remuszka_Shared/GovRnD/data/G Contracts/base_files.zip
--
Path = /home/t/ticeraskin/Desktop/project/Remuszka_Shared/GovRnD/data/G Contracts/base_files.zip
Type = zip
Physical Size = 3521570299

Everything is Ok

Files: 25
Size:       16770993250
Compressed: 3521570299
Extracted with 7za.


/tmp/ipykernel_2402580/3614927591.py:56: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(buffer)
/tmp/ipykernel_2402580/3614927591.py:56: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(buffer)
/tmp/ipykernel_2402580/3614927591.py:56: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(buffer)
/tmp/ipykernel_2402580/3614927591.py:56: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(buffer)
/tmp/ipykernel_2402580/3614927591.py:56: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(buffer)
/tmp/ipykernel_2402580/3614927591.py:56: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(

In [6]:
# Loading in the contracts from Lydia
psc_contracts = pd.read_csv(data_location + "G Contracts/ticeraskin_remuszka.csv")
psc_contracts_nonzero = psc_contracts[psc_contracts['federal_action_obligation']!=0]

/tmp/ipykernel_2402580/331931318.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  psc_contracts = pd.read_csv(data_location + "ticeraskin_remuszka.csv")


In [7]:
# Getting the contracts all together
contracts_full = pd.merge(
    contracts_nonzero,
    psc_contracts_nonzero,
    on="contract_transaction_unique_key",
    how="inner",
    suffixes=("_nonzero", "_psc")
)

In [33]:
contracts_full_nonan = contracts_full.dropna(subset=['product_or_service_code'])

RD_contracts = contracts_full_nonan[
    contracts_full_nonan['product_or_service_code'].str.startswith("A")
].copy()

print(RD_contracts.shape)

(1196155, 15)


In [154]:
RD_copy = RD_contracts.copy()   # rename as needed

# Make sure contract_date is datetime
RD_copy['action_date'] = pd.to_datetime(RD_copy['action_date'])

# Sort for deterministic behavior
RD_copy = RD_copy.sort_values(['contract_award_unique_key', 'action_date'])

# List to hold indices of rows to remove
to_remove = set()

# Loop through each key group
for key, g in RD_copy.groupby('contract_award_unique_key'):
    g = g.sort_values('action_date')
    g_idx = g.index.tolist()
    
    for i in range(len(g) - 1):
        first = g.iloc[i]
        second = g.iloc[i + 1]
        
        # Must be within 30 days
        if abs(second.action_date - first.action_date) <= pd.Timedelta(days=30):
            
            # Check offsetting rule
            if abs(first.federal_action_obligation_nonzero + second.federal_action_obligation_nonzero) < 0.05 * abs(first.federal_action_obligation_nonzero):
                to_remove.add(first.name)
                to_remove.add(second.name)

# Drop all offsetting rows
RD_clean = RD_copy.drop(index=to_remove).copy()

In [183]:
# Ensure dates are datetime
RD_clean['action_date'] = pd.to_datetime(RD_clean['action_date'])

# Sort so "first" is meaningful
RD_clean = RD_clean.sort_values(['contract_award_unique_key', 'action_date'])

# Aggregation function
RD_clean = (
    RD_clean
    .groupby('contract_award_unique_key')
    .agg(
        action_date=('action_date', 'first'),
        federal_action_obligation_nonzero=('federal_action_obligation_nonzero', 'sum'),
        product_or_service_code=('product_or_service_code', 'first'),
        awarding_agency_code=('awarding_agency_code', 'first'),
        prime_award_transaction_place_of_performance_county_fips_code=(
            'prime_award_transaction_place_of_performance_county_fips_code', 
            'first'
        )
    )
    .reset_index()
)

RD_clean = RD_clean[RD_clean['federal_action_obligation_nonzero']>0]

In [184]:
fips_walk = pd.read_csv(data_location + "Misc/ssa_fips_state_county_2025.csv")

In [241]:
# Convert both columns to strings
RD_clean["prime_award_transaction_place_of_performance_county_fips_code"] = \
    RD_clean["prime_award_transaction_place_of_performance_county_fips_code"].astype(str)

fips_walk["fipscounty"] = fips_walk["fipscounty"].astype(str)

# Optional: zero-pad to 5 digits if needed
#RD_contracts["prime_award_transaction_place_of_performance_county_fips_code"] = \
#    RD_contracts["prime_award_transaction_place_of_performance_county_fips_code"].str.zfill(5)

fips_walk["fipscounty"] = fips_walk["fipscounty"].str.zfill(5)

local_merge = pd.merge(RD_clean, fips_walk, left_on = "prime_award_transaction_place_of_performance_county_fips_code", right_on = "fipscounty")

In [252]:
psc_codes = pd.read_excel(data_location + "Misc/PSC April 2025.xlsx")

RD_describe = pd.merge(local_merge, psc_codes, left_on = "product_or_service_code", right_on = "PSC CODE")

col = 'PRODUCT AND SERVICE CODE FULL NAME (DESCRIPTION)'

# 1. Extract the type part after the dash
RD_describe['research_type'] = (
    RD_describe[col]
    .str.split('-', n=1)
    .str[1]
    .str.lower()
    .str.strip()
)

# 2. Create a single categorical column: basic / applied / other
def classify_research(x):
    if pd.isna(x):
        return "other"
    if "basic" in x:
        return "basic"
    if ("applied" in x) or ("development" in x):
        return "applied"
    return "other"

RD_describe['research_category'] = RD_describe['research_type'].apply(classify_research)

In [280]:
RD_describe = pd.merge(RD_describe, dfs['crosswalk_awarding_agency.csv'], on = "awarding_agency_code")

# Ensure action_date is datetime
RD_describe['action_date'] = pd.to_datetime(RD_describe['action_date'], errors='coerce')

# Create a monthly period
RD_describe['year_month'] = RD_describe['action_date'].values.astype('datetime64[M]')

# -----------------------------
# 1. Denominator: state × month × research_category
# -----------------------------
agg_state_month_research = (
    RD_describe
    .groupby(['state', 'year_month', 'research_category'], as_index=False)
    ['federal_action_obligation_nonzero']
    .sum()
)

# -----------------------------
# 2. Agencies we want
# -----------------------------
selected_agencies = [
    "Department of Defense",
    "National Aeronautics and Space Administration",
    "Department of Energy",
    "Department of Health and Human Services",
    "National Science Foundation"
]

# Keep only those agencies
df_agencies = RD_describe[RD_describe['awarding_agency_name'].isin(selected_agencies)].copy()

# -----------------------------
# 3. Aggregate: state × month × research_category × agency
# -----------------------------
agg_agencies = (
    df_agencies
    .groupby(['state', 'year_month', 'research_category', 'awarding_agency_name'], as_index=False)
    ['federal_action_obligation_nonzero']
    .sum()
)

# -----------------------------
# 4. Pivot → one column per agency
# -----------------------------
agency_pivot = agg_agencies.pivot_table(
    index=['state', 'year_month', 'research_category'],
    columns='awarding_agency_name',
    values='federal_action_obligation_nonzero',
    fill_value=0
).reset_index()

# Clean names
agency_pivot.columns = [
    col.replace(" ", "_").replace("-", "_").lower()
    for col in agency_pivot.columns
]

# -----------------------------
# 5. Merge everything together
# -----------------------------
final_agg = agg_state_month_research.merge(
    agency_pivot,
    on=['state', 'year_month', 'research_category'],
    how='left'
)

final_agg.fillna(0, inplace=True)

print(final_agg.head())


  state year_month research_category  federal_action_obligation_nonzero  \
0    AK 2003-02-01             basic                          151389.64   
1    AK 2003-02-01             other                           42925.00   
2    AK 2003-09-01             basic                         1186644.00   
3    AK 2003-09-01             other                          593322.00   
4    AK 2004-02-01             basic                           62700.00   

   department_of_defense  department_of_energy  \
0                    0.0                   0.0   
1                    0.0                   0.0   
2                    0.0                   0.0   
3                    0.0                   0.0   
4                    0.0                   0.0   

   department_of_health_and_human_services  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                 

In [315]:
# ---- BALANCE ----

# Unique states
states = final_agg['state'].unique()

# Global month range
start_month = final_agg['year_month'].min()
end_month   = final_agg['year_month'].max()

months = pd.period_range(start=start_month, end=end_month, freq='M').to_timestamp()

# Research categories
categories = ['basic', 'applied', 'other']

# Full Cartesian product
full_index = pd.MultiIndex.from_product(
    [states, months, categories],
    names=['state', 'year_month', 'research_category']
)

# Reindex to balance panel
RD_final = (
    final_agg
    .set_index(['state', 'year_month', 'research_category'])
    .reindex(full_index, fill_value=0)
    .reset_index()
)

RD_final.rename(columns={'federal_action_obligation_nonzero': 'federal_action_obligation'}, inplace=True)

In [316]:
# ---- CREATE BASIC / APPLIED / OTHER COLUMNS ----

# Pivot from long (one row per research_category)
# → to wide (three columns: basic, applied, other)
RD_wide = (
    RD_final
    .pivot_table(
        index=['state', 'year_month'],
        columns='research_category',
        values=['federal_action_obligation','department_of_defense',
               'department_of_health_and_human_services', 'national_aeronautics_and_space_administration',
               'department_of_energy', 'national_science_foundation'],
        aggfunc='sum',
        fill_value=0
    )
    .reset_index()
)

# Ensure missing categories appear as columns
for col in ['basic', 'applied', 'other']:
    if col not in RD_wide.columns:
        RD_wide[col] = 0

# Clean column index (pivot makes a MultiIndex)
RD_wide.columns.name = None

# This is your final panel dataset
RD_panel = RD_wide.copy()


In [317]:
# ---- CREATE BASIC / APPLIED / OTHER COLUMNS ----

RD_wide = (
    RD_final
    .pivot_table(
        index=['state', 'year_month'],
        columns='research_category',
        values=[
            'federal_action_obligation',
            'department_of_defense',
            'department_of_health_and_human_services',
            'national_aeronautics_and_space_administration',
            'department_of_energy',
            'national_science_foundation'
        ],
        aggfunc='sum',
        fill_value=0
    )
    .reset_index()
)

# Flatten column MultiIndex: "name_type"
RD_wide.columns = [
    f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
    for col in RD_wide.columns
]

# Ensure missing categories exist
for cat in ['basic', 'applied', 'other']:
    for var in [
        'federal_action_obligation',
        'department_of_defense',
        'department_of_health_and_human_services',
        'national_aeronautics_and_space_administration',
        'department_of_energy',
        'national_science_foundation'
    ]:
        colname = f"{var}_{cat}"
        if colname not in RD_wide.columns:
            RD_wide[colname] = 0

rename_map = {
    'department_of_defense': 'dod',
    'department_of_health_and_human_services': 'hhs',
    'national_aeronautics_and_space_administration': 'nasa',
    'department_of_energy': 'doe',
    'national_science_foundation': 'nsf'
}

for long, short in rename_map.items():
    for cat in ['basic', 'applied', 'other']:
        old = f"{long}_{cat}"
        new = f"{short}_{cat}"
        if old in RD_wide.columns:
            RD_wide.rename(columns={old: new}, inplace=True)

# --- Rename federal_action_obligation_* to just basic/applied/other ---
for cat in ['basic', 'applied', 'other']:
    old = f"federal_action_obligation_{cat}"
    if old in RD_wide.columns:
        RD_wide.rename(columns={old: f"tot_{cat}"}, inplace=True)



RD_panel = RD_wide.copy().rename(columns = {"state_":"state", "year_month_":"date"})

In [319]:
RD_panel.to_csv(data_location + 'G Contracts/state_panel.csv')